# Multi-layer perceptrons

### Outline
1. The MLP architecture
2. Overview of tensors
3. PyTorch implementation

## 1. Multi-layer perceptrons

### Some models we've seen so far

Suppose we have a regression of classification task, and a labelled dataset with $n$ continuous features. So, we have:
\begin{equation*}
    \textup{Input space} = \mathbb{R}^{n},
\end{equation*}
where each input vector is of the form $x = (x_1,\dotsc,x_n)$.

We've seen a few different parametric models. 

1. **Linear regression** (regression tasks):
    - Output space: $\mathbb{R}$
    - Parameter space: $\mathbb{R}^{n+1}$ (the parameters are $b\in \R$ (bias) and $w\in \R^n$ (weights)).
    - Model: $F_w(x) = w^T x = w_0 + w_1 x_1 + \dotsb + w_n x_n$